In [ ]:
import sys
sys.path.append('./deploy/driver')
import socket
import numpy as np
from deploy.driver.driver import FINNExampleOverlay
from qonnx.core.datatype import DataType

# Setup the overlay
bitfile_CNN = "deploy/bitfile/finn-accel.bit"
platform = "zynq-iodma"

#Input/output dictionary to deal with FPGA CNN
_cifar10_cnv_io_shape_dict = {
    "idt": [DataType["UINT8"]],
    "odt": [DataType["UINT8"]],
    "ishape_normal": [(1, 32, 32, 3)],
    "oshape_normal": [(1, 1)],
    "ishape_folded": [(1, 1, 32, 32, 1, 3)],
    "oshape_folded": [(1, 1, 1)],
    "ishape_packed": [(1, 1, 32, 32, 1, 3)],
    "oshape_packed": [(1, 1, 1)],
    "input_dma_name": ["idma0"],
    "output_dma_name": ["odma0"],
    "number_of_external_weights": 0,
    "num_inputs": 1,
    "num_outputs": 1,
}

accel = FINNExampleOverlay(bitfile_CNN, platform, io_shape_dict=_cifar10_cnv_io_shape_dict)


print("Initialising FPGA overlay...")
accel = FINNExampleOverlay(bitfile_CNN, platform, io_shape_dict=_cifar10_cnv_io_shape_dict)

# Code for the server
HOST = "192.168.7.2"
PORT = 9999

print(f"Starting TCP server on {HOST}:{PORT}")
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST,PORT))
    s.listen()
    print("Waiting for the RPI to connect...")
    conn,addr = s.accept()
    with conn:
        print(f"Connected by {addr}")
        while True:
            #Receive 3072 bytes
            data = conn.recv(3072)
            if not data:
                break
                
            img = np.frombuffer(data, dtype=np.uint8).reshape((1,32,32,3))
            label_index = accel.execute(img).item()
            conn.sendall(int(label_index).to_bytes(1,'big')) # 1 byte